In [33]:
%run LoserAnalysis.ipynb

In [34]:
EXACT_PROBS_FILE = 'exactHittingProbs.csv'
APPROX_PROBS_FILE = 'approxHittingProbs.csv'
TIME_LIM = 10  # in seconds

In [35]:
from itertools import product
import signal
from time import sleep  # only needed for testing

In [29]:
# Custom exception for the timeout
class TimeoutException(Exception):
    pass

# Handler function to be called when SIGALRM is received
def sigalarmHandler(signum, frame):
    # We get signal!
    raise TimeoutException()

In [30]:
# Write the header
fp = open(EXACT_PROBS_FILE, 'w')
writer = csv.writer(fp)
writer.writerow(['Initial state', 'P(Loser = Player 1)'])

fp = open(APPROX_PROBS_FILE, 'w')
writer = csv.writer(fp)
writer.writerow(['Initial state', 'P(Loser = Player 1)'])

35

In [36]:
# Explores the probabilities within the given range
def writeProbsInRange(minWealth, maxWealth):
    allStates = product(range(minWealth, maxWealth+1), repeat=NUM_PLAYERS)
    
    for state in allStates:
        # Set up signal handler for SIGALRM, saving previous value
        oldHandler = signal.signal(signal.SIGALRM, sigalarmHandler)
        # Start timer
        signal.alarm(TIME_LIM)
        try:
            exactProb = LoserAnalysis(state).getHittingProb()
            approxProb = float(LoserAnalysis(state).getHittingProb())
            exportStateProbs(EXACT_PROBS_FILE, {state:exactProb}, append=True, header=False)
            exportStateProbs(APPROX_PROBS_FILE, {state:approxProb}, append=True, header=False)
        except TimeoutException:
            print(f'Took too long to find the hitting prob for state {state}')
        finally:
            # Turn off timer
            signal.alarm(0)
            # Restore handler to previous value
            signal.signal(signal.SIGALRM, oldHandler)

In [ ]:
writeProbsInRange(1,50)